In [3]:
import cv2
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

# Function to convert latitude/longitude to EXIF-friendly format
def convert_to_degrees(value):
    degrees = int(value)
    minutes = int((value - degrees) * 60)
    seconds = (value - degrees - minutes / 60) * 3600
    return (degrees, minutes, seconds)

# Function to add GPS metadata to the image
def add_gps_data(image_path, lat, lon):
    img = Image.open(image_path)
    exif_data = img.getexif()
    gps_info = {}
    
    # Convert latitude and longitude to EXIF format
    lat_deg = convert_to_degrees(abs(lat))
    lon_deg = convert_to_degrees(abs(lon))

    gps_info[1] = 'N' if lat >= 0 else 'S'  # Latitude Ref
    gps_info[2] = lat_deg  # Latitude
    gps_info[3] = 'E' if lon >= 0 else 'W'  # Longitude Ref
    gps_info[4] = lon_deg  # Longitude

    # Write the GPS data back into the image
    exif_data[0x8825] = gps_info  # GPSInfo tag
    img.save(image_path, exif=exif_data)
    print(f"Geolocation data added to {image_path}")

# Prompt user for latitude and longitude input
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Read the first frame as the background
ret, background = cap.read()

# Convert the background to grayscale
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to the background to reduce noise
background_blur = cv2.GaussianBlur(background_gray, (21, 21), 0)

# Screen center coordinates
height, width = background.shape[:2]
center_x, center_y = width // 2, height // 2

# Flag to prevent multiple screenshots
screenshot_taken = False

def is_centered(x, y, w, h):
    """ Check if the object is near the center of the screen """
    obj_center_x = x + w // 2
    obj_center_y = y + h // 2
    # Define a region around the center for detection
    tolerance = 50
    return (center_x - tolerance < obj_center_x < center_x + tolerance) and \
           (center_y - tolerance < obj_center_y < center_y + tolerance)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to the frame to reduce noise
    gray_blur = cv2.GaussianBlur(gray, (21, 21), 0)
    
    # Compute the absolute difference between the background and the current frame
    diff = cv2.absdiff(background_blur, gray_blur)
    
    # Apply a threshold to get the foreground mask
    _, mask = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
    
    # Invert the mask to get the background mask
    mask_inv = cv2.bitwise_not(mask)
    
    # Create a white background of the same size as the frame
    white_background = np.full_like(frame, 255)
    
    # Use the mask to extract the moving object from the current frame
    foreground = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Use the inverse mask to extract the background from the white background
    white_bg = cv2.bitwise_and(white_background, white_background, mask=mask_inv)
    
    # Combine the foreground (moving object) with the white background
    result = cv2.add(foreground, white_bg)
    
    # Find contours to detect the moving object
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Iterate over the contours to draw bounding boxes
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small objects
            x, y, w, h = cv2.boundingRect(contour)
            # Draw a bounding box around the object
            cv2.rectangle(result, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Check if the object is passing through the center
            if is_centered(x, y, w, h) and not screenshot_taken:
                screenshot_taken = True

                # Get the center of the bounding box
                obj_center_x = x + w // 2
                obj_center_y = y + h // 2
                
                # Calculate the crop area for 400x400 image centered on the bounding box center
                crop_x = max(0, obj_center_x - 200)
                crop_y = max(0, obj_center_y - 200)
                crop_x2 = min(width, crop_x + 400)
                crop_y2 = min(height, crop_y + 400)
                
                # Ensure the crop is exactly 400x400
                if (crop_x2 - crop_x) < 400:
                    crop_x = max(0, crop_x2 - 400)
                if (crop_y2 - crop_y) < 400:
                    crop_y = max(0, crop_y2 - 400)
                
                # Crop the image around the center of the bounding box
                cropped_result = result[crop_y:crop_y2, crop_x:crop_x2]
                
                # Save the cropped screenshot
                image_path = 'detected_object_cropped.png'
                cv2.imwrite(image_path, cropped_result)
                print("Cropped screenshot saved!")

                # Add geolocation metadata to the image
                add_gps_data(image_path, latitude, longitude)

    # Show the result
    cv2.imshow('Moving Object with White Background', result)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
cap.release()
cv2.destroyAllWindows()


Cropped screenshot saved!
Geolocation data added to detected_object_cropped.png
